## Filtros

In [12]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from skimage.util import random_noise

Using matplotlib backend: Qt5Agg


#### Filtro Gaussiano

In [13]:
# Parámetros:
# ksize - Tamaño del núcleo, positivo impar (3,5,...)
# sigma - Desvío estándar de la Gaussiana. Si no es positivo, se calcula como sigma = 0.3*((ksize-1)*0.5 - 1) + 0.8 
# ktype - Tipo de coeficientes del filtro (Optional). Puede ser CV_32F o CV_64F
kernel_1d = cv.getGaussianKernel(ksize=5, sigma=1.5) 
kernel_2d = kernel_1d * kernel_1d.T
print(kernel_1d)
print(f'ganancia: {np.sum(kernel_2d)}')
print(kernel_2d)

[[0.12007838]
 [0.23388076]
 [0.29208172]
 [0.23388076]
 [0.12007838]]
ganancia: 1.0
[[0.01441882 0.02808402 0.0350727  0.02808402 0.01441882]
 [0.02808402 0.05470021 0.06831229 0.05470021 0.02808402]
 [0.0350727  0.06831229 0.08531173 0.06831229 0.0350727 ]
 [0.02808402 0.05470021 0.06831229 0.05470021 0.02808402]
 [0.01441882 0.02808402 0.0350727  0.02808402 0.01441882]]


In [9]:
# Suavizado Gaussiano usando el Kernel
#=====================================
img_orig = cv.imread('chewbacca.jpg')
 
# Creamos un núcleo Gaussiano unidimensional
kX = cv.getGaussianKernel(ksize=17, sigma=5)
kY = cv.getGaussianKernel(ksize=17, sigma=5)
 
# Aplicamos el filtro Gaussiano. Con distinto núcleo para X e Y!
result_a = cv.sepFilter2D(img_orig,  ddepth=-1, kernelX=kX, kernelY=kY)

# Más fácil utilizando una única línea
# (cuando el kernel es cuadrado)

result_b = cv.GaussianBlur(img_orig, ksize=(17, 17), sigmaX=5,  sigmaY=5)

# Mostramos la imagen
out1 = cv.hconcat([result_b, result_a])


cv.imshow("Chewbacca - sepFilter2D", out1)
cv.imshow("Chewbacca - GaussianBlur", img_orig)

#### Filtro de mediana



In [10]:
img2 = cv.imread('tapitas.jpg',0)

# Agregar ruido salt&pepper a la imagen
noise_img = random_noise(img2, mode="s&p", amount=0.3)
noise_img = np.array(255*noise_img, dtype = 'uint8')

# Otra manera de agregar ruido (no es salt&pepper) con numpy
# gauss = np.random.normal(0,1,img2.size)
# gauss = gauss.reshape(img2.shape[0],img2.shape[1]).astype('uint8')
# noise_img = img2 + img2 * gauss

# Aplicar el filtro de media
mean = cv.blur(noise_img, (5,5))

# Aplicar el filtro de mediana
median = cv.medianBlur(noise_img, 5)
 
# Mostrar la imagen
cv.imshow('Original - S&P', noise_img)
cv.imshow('Media', mean)
cv.imshow('Mediana', median)

#### DoG

In [ ]:
# Cargamos la imagen
img_orig = cv.imread('tapitas.jpg',0)
#img_orig = cv.imread('chewbacca.jpg',0)

# Aplicamos un suavizado Gaussiano de 5x5
low_sigma = cv.GaussianBlur(img_orig, (5,5), sigmaX=1)
high_sigma = cv.GaussianBlur(img_orig, (5,5),sigmaX=1.5)

# Calculamos el DoG por sustracción
dog = low_sigma - high_sigma

# Aplico filtro de mediana para mejorar el resultado
median = cv.medianBlur(dog, 5)

# Mostramos la imagen
out1 = cv.hconcat([img_orig, dog, median])
cv.namedWindow("DoG", 2)
cv.imshow("DoG",out1)

#### Unsharp masking

In [11]:
# Cargar la imagen (la carga como BGR)
img_orig = cv.imread("chewbacca.jpg")
# La convertimos a RGB
img_orig = cv.cvtColor(img_orig, cv.COLOR_BGR2RGB)
# Suavizar la imagen
gauss = cv.GaussianBlur(img_orig, (7,7), 0.5)
# Aplicar la máscara de enfoque: 
# addWeighted calcula la suma pesada de dos arreglos según: dst(I)=saturate(src1(I)∗alpha+src2(I)∗beta+gamma)
# parámetros: (src1(I),alpha,src2(I),beta,gamma)
img_sharp = cv.addWeighted(img_orig, 2, gauss, -1, 0)

# Más enfocada
k=1.5
img_sharp2 = cv.addWeighted(img_orig, k+1, gauss, -k, 0)

# Re enfocada
k=50
img_sharp3 = cv.addWeighted(img_orig, k+1, gauss, -k, 0)


# Nueva figura
fig = plt.figure()

ax1=plt.subplot(221)
ax1.set_title('Original')
ax1.imshow(img_orig)
ax2=plt.subplot(222)
ax2.set_title('Enfocada (k=1)')
ax2.imshow(img_sharp)
ax3=plt.subplot(223)
ax3.set_title('Más enfocada (k=1.5)')
ax3.imshow(img_sharp2)
ax4=plt.subplot(224)
ax4.set_title(f'Re enfocada {k}')
ax4.imshow(img_sharp3)

fig.show()